In [1]:
%%writefile lab/PSO.cpp

#include <mpi.h>
#include <iostream>
#include <omp.h>
#include <cstdlib>
#include <time.h>
#include <random>
#include <thread>
#include <math.h>
using namespace std;

constexpr int kMaster = 0;
const int n_dim = 6;
const int n_particles = 16000;
const double solution_space = 15;
const int iters = 5000;
const double w = 0.8;
const double c1 = 0.8;
const double c2 = 0.8;


double rand_num(double min = 0.0, double max = 1.0, int seed = 0);
double search_parrallel(double** x, double** v, double* pbest, double* gbest, int k);


double func(double x[])
{
    double y;
    y = pow((x[0] - 2), 2) + pow((x[1] - 5), 2) + pow((x[2] - 8), 2) + pow((x[3] + 3.2425), 2) + pow((x[4] - 2.43), 2) + pow((x[5] - 3), 2);
    return y;
}


int main(int argc, char* argv[]) {
    const clock_t begin_time = clock();
    
    
    //initialization
    double** x = new double* [n_particles];
    double** v = new double* [n_particles];
    for (int i = 0; i < n_particles; i++)
    {
        x[i] = new double[n_dim];
        v[i] = new double[n_dim];
        for (int j = 0; j < n_dim; j++)
        {
            x[i][j] = rand_num(-solution_space, solution_space);
            v[i][j] = rand_num();
        }
    }
    double pbest_f = func(x[0]), gbest_f = func(x[0]);
    double* pbest = new double[n_dim];
    double* gbest = new double[n_dim];

    
    //start search
    for (int k = 1; k <= iters; k++)
    {
        //particles move and get best of current iteration
        gbest_f = search_parrallel(x, v, pbest, gbest, k);
        //get best of current iteration
        if (1)
        {
            //get best of all iterations
            if (gbest_f < pbest_f)
            {
                pbest_f = gbest_f;
                copy(gbest, gbest + n_dim, pbest);
            }
        }
    }
    
    
    //print answer
    if (1)
    {
        cout << "solution : [";
        for (int j = 0; j < n_dim; j++)
        {
            if (j!=0)
                cout << ", ";
            cout << pbest[j];
        }
        cout << "]" << endl;
        cout << pbest_f << endl;
        cout << "Time spent: " << float( clock () - begin_time ) /  CLOCKS_PER_SEC << " secs" << endl;
    }

    
    //free memory
    for (int i = 0; i < n_particles; i++)
    {
        delete[] x[i];
        delete[] v[i];
    }
    delete[] x;
    delete[] v;
    delete[] pbest;
    delete[] gbest;

    return 0;
}

double rand_num(double min, double max, int seed)
{
    static thread_local mt19937 generator(time(0) + seed);
    //static thread_local mt19937 generator;
    uniform_real_distribution<double> distribution(min, max);
    return distribution(generator);
}

double search_parrallel(double** x, double** v, double* pbest, double* gbest, int k)
{
    //particles relocate
    double new_gbest_f;
    int new_gbest_id;

    for (int i = 0; i < n_particles; i++)
    {
        double r1 = rand_num(), r2 = rand_num();
        for (int j = 0; j < n_dim; j++)
        {
            if (k == 1)
            {
                //pbest[j] = x[i][j];
                //gbest[j] = x[i][j];
                v[i][j] = w * v[i][j];
            }
            else
            {
                v[i][j] = w * v[i][j] + c1 * r1 * (pbest[j] - x[i][j]) + c2 * r2 * (gbest[j] - x[i][j]);
            }
            x[i][j] += v[i][j];
        }
        // calculate gbest in same loop
        if ((i == 0) || (func(x[i]) < new_gbest_f))
        {
            new_gbest_f = func(x[i]);
            new_gbest_id = i;
        }
    }
    /*for (int i = 0; i < n_particles; i++)
    {
        if ((i == 0) || (func(x[i]) < new_gbest_f))
        {
            new_gbest_f = func(x[i]);
            new_gbest_id = i;
        }
    }*/
    copy(x[new_gbest_id], x[new_gbest_id] + n_dim, gbest);
    return new_gbest_f;
}

Overwriting lab/PSO.cpp


In [2]:
%pycat compile_omp_c.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
/bin/echo "##" $(whoami) is compiling
mpiicpc -cxx=icpx lab/PSO.cpp -fiopenmp -fopenmp-targets=spir64 -o bin/PSO.x


The following cell will submit the execution of the compilation script using the __q__ script. The __q__ script submits jobs to the DevCloud and retrieves the output. The first arguments to __q__ is the script to execute. The second argument is the properties of the nodes to request. In the following cell, we're requesting one node with the property ppn=2.

In [3]:
! chmod 755 q; chmod 755 compile_omp_c.sh; ./q compile_omp_c.sh nodes=1:ppn=2

Job has been submitted to Intel(R) DevCloud with  nodes and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2182726.v-qsvr-1           ...ub-singleuser u184460         00:04:05 R jupyterhub     
2183803.v-qsvr-1           compile_omp_c.sh u184460                0 Q batch          

Waiting for Output ███████████████████ Done⬇

########################################################################
#      Date:           Thu 09 Feb 2023 07:56:10 AM PST
#    Job ID:           2183803.v-qsvr-1.aidevcloud
#      User:           u184460
# Resources:           cput=75:00:00,neednodes=1:ppn=2,nodes=1:ppn=2,walltime=06:00:00
########################################################################

## u184460 is compiling

##

### Execute the Code
Next we will execute the compiled binary with the __mpirun__ command to launch the MPI job using 4 processes. Examine the launch script by executing the following cell.

In [4]:
%pycat launch.sh

#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1
% setenv OMP_TARGET_OFFLOAD MANDATORY
export IBOMPTARGET_INFO=4
/bin/echo "##" $(whoami) is executing
mpirun -np 4 bin/PSO.x


Execute the following cell to run the program on multiple nodes. In the following example, 2 nodes with GPUs are requested.

In [5]:
! chmod 755 q; chmod 755 launch.sh; ./q launch.sh nodes=2:gpu:ppn=2

Job has been submitted to Intel(R) DevCloud with  nodes and will execute soon.

 If you do not see result in 60 seconds, please restart the Jupyter kernel:
 Kernel -> 'Restart Kernel and Clear All Outputs...' and then try again

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2182726.v-qsvr-1           ...ub-singleuser u184460         00:04:05 R jupyterhub     
2183804.v-qsvr-1           launch.sh        u184460                0 Q batch          

Waiting for Output █████████████████████████ Done⬇

########################################################################
#      Date:           Thu 09 Feb 2023 07:56:29 AM PST
#    Job ID:           2183804.v-qsvr-1.aidevcloud
#      User:           u184460
# Resources:           cput=75:00:00,neednodes=2:gpu:ppn=2,nodes=2:gpu:ppn=2,walltime=06:00:00
########################################################################

## u184460 is